In [ ]:
#깃허브의 1일차-> data.zip
#creditset2.csv -> 데이터프레임
# 1. default컬럼 선택 1->"d", 0->"n"
# 2. 1~1400행 선택, 1401행~2000행
# 3. LTI를 삭제

import pandas as pd
df = pd.read_csv("creditset2.csv", header=None , index_col=[0] , parse_dates= [1] )
#df.info()
df1 = df.iloc[1:2001,    ]  #
df1["col4"] = df1.index
df1.reset_index(inplace=True)
df1 = df1.iloc[ :  ,  1:5]
df1.columns = ["age", "loan", "default", "income"]  #Loan To Income
df1.age
df1.iloc[:, 0]

df1["age"]     #1D
df1[    ["age" ]    ]     #2D 2000행 1열짜리,  df1[    ["age", "income" ]    ]

df1["age"] = df1.age.astype("float")  #실수
#loan, income dtype을 변환해서 LTI컬럼 추가
df1["loan"] = df1.loan.astype("float")  #실수
df1["income"] = df1.income.astype("float")  #실수
df1["LTI"] = df1.loan / df1.income  #LTI
df1.LTI.tolist()   #list

In [ ]:
df1.default.value_counts()
df1.default.replace( ["0","1"]      ,  ["N", "D"]      )
df1.default.replace( {"0":"N", "1":"D"}    )   # inplcae=True

def func1( x ):
  if x == "0":
    return "N"
  elif x== "1":
    return "D"
  else:
    return "ERR"

df1.default.apply( func1 )   #df1.default.apply( lambda x: "N" if x=="0" else "D" if x=="1" else "ERR" )


In [ ]:
#creditset2.csv를 df2로 읽으세요
#default가 1인 경우 다른 세 개변수 평균, 0인 경우의 다른 세개변수 평균
df2 = pd.read_csv("creditset2.csv")

df2[ df2.default10yr==0  ].drop("default10yr", axis=1).mean()
df2[ df2.default10yr==1  ].drop("default10yr", axis=1).mean()

df2.groupby("default10yr").mean()

#binary.csv ->admit이 1인 경우 합격, 0인경우는 불합격
#합격생 GRE, GPA 평균과, 불합격생의 ...을 비교

df3 = pd.read_csv("binary.csv")
df3.groupby("admit").mean().drop("rank", axis=1)

In [ ]:
#creditapprovaluci.csv
#df4 ->수치형 dtype 컬럼과, object형 dtype 컬럼들을 각각 선택
#df4_1, df4_2
df4 = pd.read_csv("creditApprovalUCI.csv")
df4_1 = df4.select_dtypes(include="number")
df4_2 = df4.select_dtypes(include="object")
df4.shape, df4_1.shape, df4_2.shape
df4_1.isnull().mean()
df4_2.dropna().shape

#imputing
df4_1.fillna( df4_1.mean(), inplace=True)   #각 변수의 결측치를, 각 변수의 평균으로 ...
#df4_2.fillna(     df4_1.mean(), inplace=True)   #각 변수의 결측치를, 각 변수의 평균으로 ...
df4_2.fillna("ERR")

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="most_frequent") #imputer.fit(df4_2)  #imputer.statistics_
df4_2=pd.DataFrame(imputer.fit_transform( df4_2 ), columns=df4_2.columns, index=df4_2.index )

pd.concat( [ df4_1  , df4_2  ]    , axis=1     )
df4_3 = df4_1.merge( df4_2, left_index=True, right_index=True )  #left_on, right_on


In [ ]:
#AMZN.csv를 읽으셔서 df5, date? 컬럼을 datetime64로 지정
#데이터프레임을 Close, Volume컬럼을 갖는 두 데이터프레임으로 나눠보세요.
#두 데이터프레임을 하나로 합쳐보세요.

df5 = pd.read_csv("AMZN.csv", parse_dates=[0], index_col=[0])
df5.head()
df5_1 = df5[ ["Volume"] ]
df5_2 = df5[ ["Close"] ]
pd.concat( [ df5_1, df5_2 ] , axis=1 )
df5_3 = df5_1.merge(df5_2, left_index=True, right_index=True)
df5_3.reset_index(inplace=True)
df5_3.Date.dt.dayofweek
df5_3.Date.dt.day_of_year
df5_3.Date.dt.quarter

#AMZN-> 분기별로 Close, Volume 평균
df5["qt"] = df5.reset_index().Date.dt.quarter.values
df5[["qt", "Close", "Volume"]].groupby("qt").mean()
#Date -> object -> "-"를 ":"으로 변경해보세요
df5.reset_index().Date.astype("object")#.replace("\-", "\:")

#아레 3개 데이터프레임을 하나로 합쳐보세요 -> df5_4
df5_4 = pd.concat( [ df5[["Close"]], #t
 df5[["Close"]].shift(1),  #t-1
df5[["Close"]].shift(2) #t-2
 ], axis=1)
df5_4.columns = ["Close","Close(t-1)", "Close(t-2)"]
df5_4.head()
df5_4.dropna(inplace=True)
df5_4.Close.diff()
df5_4.Close.pct_change()

In [ ]:
#sahmpoo.txt을 읽으셔서 일자별  판매량의 차이와 변화율을 구해보세요,.
df6 = pd.read_csv("shampoo.txt")
df6.Sales.diff().dropna()
df6.Sales.pct_change().dropna()

In [ ]:
!pip install pykrx -qqq

In [ ]:
from pykrx import stock
df7=stock.get_market_ohlcv_by_date(ticker="005930",fromdate="20250701", todate="20250730")
df7.거래량.pct_change()
df7_1 = pd.concat( [df7.종가, df7.종가.shift(1), df7.종가.shift(2)], axis=1 )


In [ ]:
df7.종가   #63000보다 큰 행
price = 63000
df7.query( "( 종가 > @price and 등락률>0 ) or 거래량 > 20000000"   )
df7.query( "( 종가 > @price and 등락률>0 ) or 거래량 != '없음'"   )

In [ ]:
df3.query(  " rank not in [3,4]")

In [ ]:
import seaborn as sns
df8 = sns.load_dataset("diamonds")
df8.cut.unique()
df8.clarity.unique()
#df8에서 가격은 평균 이상이고, cut이 fair 또는 good 인 행
avg = df8.price.mean()
df8.query( "price >=@avg and cut in ['Fair', 'Good'] ").to_csv("123.csv")
#df8에서 수치형 변수만 선택-> corr()
df8_1 = df8.select_dtypes(include="number")
#sns.heatmap( df8_1.corr())
#sns.pairplot(df8_1)
#sns.jointplot(data=df8_1, x="carat", y="price")

In [ ]:
df9 = sns.load_dataset("tips")
#total_bill, tip에 대한 joint plot
#sns.jointplot( df9, x="total_bill", y="tip")
#요일별 total_bill, tip의 평균->heatmap
#sns.heatmap( df9[["total_bill", "tip", "day"]].groupby("day").mean()  )

df9[["total_bill", "tip"]].corr()  #correlation 상관관계, 두 변수의 선형관계를 알려줌.
#-1~1, 0: 선형관계없음.

,total_bill,tip
total_bill,1.000000,0.675734
tip,0.675734,1.000000


# 회귀: Regression
- 영향을 주는 변수(들) (수치, 범주형, 여러개):
X
- 영향을 받는 변수(수치, 1개): y


In [ ]:
#삼성전자 주가를 6개월, 수집
df10=stock.get_market_ohlcv_by_date(ticker="005930",fromdate="20250201",todate="20250730")

In [ ]:
# Close, Close(t-1), Close(t-2), 거래량(t-1) 데이터프레임
# y, X1, X2, X3
df10_1 = pd.concat( [df10.종가,
                     df10.종가.shift(1),
                     df10.종가.shift(2),
                     df10.거래량.shift(1)
                     ],
                    axis=1 )
df10_1.columns = ["y", "X1", "X2", "X3"]
df10_1.dropna(inplace=True)

In [ ]:
df10_1.drop("y", axis=1).tail(1).shape
df10_1.shape

(120, 4)

In [ ]:
#adv.csv -> sales->y변수, tv, news, radio -> X
from sklearn.tree import DecisionTreeRegressor #ML, Regression
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df10_1.drop('y', axis=1),
                                                    df10_1.y,
                                                    train_size=0.8)
model = RandomForestRegressor()
model.fit( X_train, y_train  )     #
pred = model.predict(  X_test )

import numpy as np
from sklearn.metrics import mean_squared_error
np.sqrt( mean_squared_error( y_test, pred) )  #((y_test - pred)**2).mean()**0.5   #rooted mean squared error  RMSE, MSE

np.float64(870.5020582016641)

In [ ]:
#adv.csv -> sales->y변수, tv, news, radio -> X
adv = pd.read_csv("adv.csv", index_col=[0])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(adv.drop('sales', axis=1),#X
                                                    adv.sales,
                                                    train_size=0.8,
                                                    random_state=1)
model = RandomForestRegressor(n_estimators=100, random_state=1)
model.fit( X_train, y_train  )     #
pred = model.predict(  X_test )

np.sqrt( mean_squared_error( y_test, pred) )

np.float64(0.6824936629742423)

In [ ]:
#1. 다음 두 함수를 이용해서 같은 기간에 대한 같은 회사의 데이터를 수집
from pykrx import stock
stock.get_market_fundamental_by_date(ticker="005930",fromdate="20250201",todate="20250730")
stock.get_market_ohlcv_by_date(ticker="005930",fromdate="20250201",todate="20250730")

#2. 위의 데이터프레임을 하나로 합치세요
from sklearn.model_selection import train_test_split

#3. 종가를 y변수로 하고, 파티셔닝을 한 후 회귀 모형을 만들어보세요.
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
#.fit(  ), .predict( )
#4. 성능을 측정해보세요. RMSE
from sklearn.metrics import mean_squared_error
import numpy as np
np.sqrt( mean_squared_error(          ))

In [278]:
#mortDefault.zip을 업로드 하고, 아래 코드를 통해서 압축을 해제하세요
!unzip mortDefault.zip

Archive:  mortDefault.zip
  inflating: mortDefault2007.csv     
  inflating: mortDefault2008.csv     
  inflating: mortDefault2009.csv     


In [290]:
#mortDefault2007.csv 를 읽은 후, default가 1인 행들을 선택하세요
data = pd.read_csv("mortDefault2007.csv")
mort = data.query("default==1")

In [296]:
#creditScore를 y로, houseAge, yearsEmployed, ccDebt을 X변수로 하고, 파티셔닝해서 모델링하세요
X_train, X_test, y_train, y_test = train_test_split(mort[["houseAge", "yearsEmploy", "ccDebt"]],
                                                    mort.creditScore,
                                                    train_size=0.8,
                                                    random_state=1)
model = RandomForestRegressor(n_estimators=200, random_state=1)
model.fit( X_train, y_train  )     #
pred = model.predict(  X_test )


In [297]:
#성능을 측정해보세요.
np.sqrt( mean_squared_error( y_test, pred) )

np.float64(50.595318904228904)